Imports

In [72]:
import cv2
import numpy as np
import os

In [73]:
show_steps = False

Constants

In [74]:
# Intrinsic Matrix
K = np.array([[2564.3186869, 0, 0], [0, 2569.70273111, 0], [0, 0, 1]])

# Circle Radius
R = 10.0

Function

In [75]:
def find_most_circular_contour(contours):

    best_circularity = 0
    best_contour = None
    best_index = -1
    
    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        
        if perimeter > 0 and area > 500:  
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            
            if circularity > best_circularity:
                best_circularity = circularity
                best_contour = contour
                best_index = i
    
    return best_contour, best_circularity, best_index

In [76]:
def calculate_3d_coordinates(contours, centers, camera_matrix, circle_radius):

    coordinates_3d = []
    reference_depth = None
    
    # Extract camera parameters
    fx = camera_matrix[0, 0]
    fy = camera_matrix[1, 1]
    cx = camera_matrix[0, 2]
    cy = camera_matrix[1, 2]
    
    # Find the most circular contour
    circle_contour, circularity, circle_index = find_most_circular_contour(contours)
    
    if circle_contour is not None and circularity > 0.8:

        # Calculate depth from the most circular object
        area = cv2.contourArea(circle_contour)
        radius_pixels = np.sqrt(area / np.pi)
        
        # Calculate reference depth
        reference_depth = (fx * circle_radius) / radius_pixels
            
    else:

        # Default depth
        reference_depth = 100.0 
    
    # Calculate 3D coordinates for all shapes using reference depth
    for i, (contour, center) in enumerate(zip(contours, centers)):

        # Image coordinates
        u, v = center  
        
        # Calculate 3D coordinates
        Z = reference_depth
        X = (u - cx) * Z / fx
        Y = (v - cy) * Z / fy
        coordinates_3d.append((X, Y, Z))
    
    return coordinates_3d, circle_index

In [77]:
def draw_3d_annotations(frame, contours, centers, coordinates_3d, circle_index=-1):
    
    result_frame = frame.copy()
    
    # Draw contours
    cv2.drawContours(result_frame, contours, -1, (0, 255, 0), 2)
    
    for i, (contour, center, coord_3d) in enumerate(zip(contours, centers, coordinates_3d)):
        cx, cy = center
        X, Y, Z = coord_3d
        
        # Get bounding box for text placement
        x, y, w, h = cv2.boundingRect(contour)
        
        # Draw center
        cv2.circle(result_frame, center, 5, (255, 255, 255), -1)

        # Place 3D coordinates
        text_3d = f"({X:.1f},{Y:.1f},{Z:.1f})"
        cv2.putText(result_frame, text_3d, (cx - 80, y + h + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    return result_frame

Load Image

In [78]:
file_dir = "data"
file_name = "PennAir 2024 App Static.png"
file = os.path.join(file_dir, file_name)

In [79]:
# Load image
img = cv2.imread(file)

In [80]:
img = cv2.resize(img, (1920, 1080))

In [81]:
width = img.shape[1]
height = img.shape[0]

In [82]:
if show_steps:
    # Display original image
    cv2.imshow("Original", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

Update K

In [83]:
# Update camera matrix principal point if not provided
# (Usually cx = width/2, cy = height/2)
K_updated = K.copy()
if K_updated[0, 2] == 0:  # If cx not set
    K_updated[0, 2] = width / 2
if K_updated[1, 2] == 0:  # If cy not set
    K_updated[1, 2] = height / 2

Binary Value (Brightness)

In [84]:
# Convert to Hue Saturation Value format
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

In [85]:
# Extract only Value (Brightness)
val_img = hsv_img[:,:,2]

# Set brightness threshold x
x = 140
_, bin_val_img = cv2.threshold(val_img, x, 255, cv2.THRESH_BINARY)

In [86]:
if show_steps:
    # Display binary value image
    cv2.imshow("Binary Value (Brightness)", bin_val_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [87]:
# Clean up w/ morphological operations
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

# Remove noise
smoothed_img = cv2.morphologyEx(bin_val_img, cv2.MORPH_OPEN, kernel)

# Fill gaps + smooth edges
smoothed_img = cv2.morphologyEx(smoothed_img, cv2.MORPH_CLOSE, kernel)

# Smooth edges again
smoothed_img = cv2.medianBlur(smoothed_img, 5)

In [88]:
if show_steps:
    # Display smoothed binary value image
    cv2.imshow("Smoothed Binary Value (Brightness)", smoothed_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

Find/Draw Outlines and Centers

In [89]:
contours, _ = cv2.findContours(smoothed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [90]:
# Remove noise
filtered_contours = []
for contour in contours:
    area = cv2.contourArea(contour)
    # How much noise to filter
    if area > 500:
        filtered_contours.append(contour)

In [91]:
# Calculate centers of contours
centers = []
for contour in filtered_contours:
    # Calculate moments
    M = cv2.moments(contour)
    if M["m00"] != 0: 
        cx = int(M["m10"] / M["m00"])  # x coord
        cy = int(M["m01"] / M["m00"])  # y coord
        centers.append((cx, cy))

In [92]:
# Copy original image to draw on
result = img.copy()

# Calculate 3D coordinates and get index in list of contours of most circular object
coordinates_3d, circle_index = calculate_3d_coordinates(filtered_contours, centers, K_updated, R)

# Draw results with 3D annotations
result = draw_3d_annotations(result, filtered_contours, centers, coordinates_3d, circle_index)

In [93]:
if show_steps:
    # Display Result
    cv2.imshow("Result", result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

Save Annotated Image to Output Folder

In [94]:
output_dir = os.path.join("output", "brightness")
output_name = f"annotated_3D_{file_name}"
output = os.path.join(output_dir, output_name)

In [95]:
cv2.imwrite(output, result)

True